In [96]:
import pandas as pd
import numpy as np
from gensim import corpora , models , similarities
import nltk

## Import data

In [113]:
df_jobs = pd.read_csv('Data_Recommdations_16sept2022/Final_JobsData1(Naukri).csv')

In [114]:
df_jobs.head()

,Job Title,Skills,location,Exp.,Role,Salary,Job Description,Job Highlights,Org. Name,url
0,Staff Software Engineer,"gcp,aws",Bangalore/Bengaluru,7 - 10 years,Back End Developer,Not Disclosed,Staff Software Engineer,"As a successful candidate,you must have a deep...",Twilio Technology India Private Limited,https://www.naukri.com/job-listings-staff-soft...
1,Job Opportunity: Senior Software Engineer,"reliability,scalability,design patterns,java,w...",Gurgaon/Gurugram,4 - 7 years,Full Stack Developer,Not Disclosed,NaN,NaN,Cvent,https://www.naukri.com/job-listings-opportunit...
2,Senior Software Engineer,"sql,oracle,dba,aix",Bangalore/Bengaluru,5 - 10 years,Embedded Systems Engineer,Not Disclosed,NaN,Should have experience & ability to work with ...,Mphasis,https://www.naukri.com/job-listings-senior-sof...
3,Senior Software Engineer C#.Net,"pdf,microservices",Trivandrum/Thiruvananthapuram,4 - 8 years,Full Stack Developer,Not Disclosed,"Skill Set: C#, Angular, OOPS",Should have Experience in Web API. The candida...,Macrosoft IT Solutions India,https://www.naukri.com/job-listings-senior-sof...
4,Team Lead and Senior Software Engineer | PHP,"php,react.js,postgresql,codeigniter,web applic...",Kochi/Cochin,4 - 9 years,Other,Not Disclosed,"At Fingent, we believe that what we build touc...",NaN,Fingent,https://www.naukri.com/job-listings-team-lead-...


In [85]:
df_jobs.describe()

,Job Title,Skills,location,Exp.,Role,Salary,Job Description,Job Highlights,Org. Name,url
count,4981,4981,4981,4981,4981,4981,3308,2721,4854,4981
unique,2243,2837,132,101,112,96,2166,2415,1671,4970
top,Senior Software Engineer,python,Bangalore/Bengaluru,3 - 6 years,Back End Developer,Not Disclosed,or,"Alongwith,the role also requires very good ana...",Varite,https://www.naukri.com/job-listings-sr-softwar...
freq,808,154,1488,358,2047,4798,113,15,225,2


In [115]:
df_tmp = df_jobs.drop(['Salary','Job Description','Job Highlights'],axis=1)

In [116]:
df = pd.DataFrame(df_tmp)

In [117]:
df

,Job Title,Skills,location,Exp.,Role,Org. Name,url
0,Staff Software Engineer,"gcp,aws",Bangalore/Bengaluru,7 - 10 years,Back End Developer,Twilio Technology India Private Limited,https://www.naukri.com/job-listings-staff-soft...
1,Job Opportunity: Senior Software Engineer,"reliability,scalability,design patterns,java,w...",Gurgaon/Gurugram,4 - 7 years,Full Stack Developer,Cvent,https://www.naukri.com/job-listings-opportunit...
2,Senior Software Engineer,"sql,oracle,dba,aix",Bangalore/Bengaluru,5 - 10 years,Embedded Systems Engineer,Mphasis,https://www.naukri.com/job-listings-senior-sof...
3,Senior Software Engineer C#.Net,"pdf,microservices",Trivandrum/Thiruvananthapuram,4 - 8 years,Full Stack Developer,Macrosoft IT Solutions India,https://www.naukri.com/job-listings-senior-sof...
4,Team Lead and Senior Software Engineer | PHP,"php,react.js,postgresql,codeigniter,web applic...",Kochi/Cochin,4 - 9 years,Other,Fingent,https://www.naukri.com/job-listings-team-lead-...
...,...,...,...,...,...,...,...
4976,Software Engineer III,"computer science,python",Hyderabad/Secunderabad,4 - 9 years,Back End Developer,EA Games (Electronic Arts),https://www.naukri.com/job-listings-software-e...
4977,Software Engineer [Backend - Python/GoLang],"computer science,python",Mumbai,3 - 5 years,Back End Developer,Egnyte,https://www.naukri.com/job-listings-software-e...
4978,Software Engineer III,"computer science,python",Hyderabad/Secunderabad,3 - 5 years,Back End Developer,FACTSET SYSTEMS INDIA PVT LTD,https://www.naukri.com/job-listings-software-e...
4979,Software Engineer .NET,".NET, C#, Angular,Html,css",Pune,3 - 6 years,Software Development - Other,Data Axle,https://www.naukri.com/job-listings-300522507556


In [123]:
df['text'] = df["Job Title"].map(str)+" "+df['Skills'].map(str)+" "+df['location'].map(str)+" "+df['Exp.'].map(str)+" "+df['Role'].map(str)+" "+df['Org. Name'].map(str)
df.head()

,Job Title,Skills,location,Exp.,Role,Org. Name,url,text
0,Staff Software Engineer,"gcp,aws",Bangalore/Bengaluru,7 - 10 years,Back End Developer,Twilio Technology India Private Limited,https://www.naukri.com/job-listings-staff-soft...,"Staff Software Engineer gcp,aws Bangalore/Beng..."
1,Job Opportunity: Senior Software Engineer,"reliability,scalability,design patterns,java,w...",Gurgaon/Gurugram,4 - 7 years,Full Stack Developer,Cvent,https://www.naukri.com/job-listings-opportunit...,Job Opportunity: Senior Software Engineer reli...
2,Senior Software Engineer,"sql,oracle,dba,aix",Bangalore/Bengaluru,5 - 10 years,Embedded Systems Engineer,Mphasis,https://www.naukri.com/job-listings-senior-sof...,"Senior Software Engineer sql,oracle,dba,aix Ba..."
3,Senior Software Engineer C#.Net,"pdf,microservices",Trivandrum/Thiruvananthapuram,4 - 8 years,Full Stack Developer,Macrosoft IT Solutions India,https://www.naukri.com/job-listings-senior-sof...,"Senior Software Engineer C#.Net pdf,microservi..."
4,Team Lead and Senior Software Engineer | PHP,"php,react.js,postgresql,codeigniter,web applic...",Kochi/Cochin,4 - 9 years,Other,Fingent,https://www.naukri.com/job-listings-team-lead-...,Team Lead and Senior Software Engineer | PHP p...


In [126]:
df_all = df[['Job Title','text','url']]

In [127]:
df_all['text'][0]

'Staff Software Engineer gcp,aws Bangalore/Bengaluru 7 - 10 years Back End Developer Twilio Technology India Private Limited'

## Download NLTK packages

In [97]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\Piyush
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to C:\Users\Piyush
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to C:\Users\Piyush
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Piyush Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

## Data Preprocessing

In [101]:
from nltk.corpus import stopwords
import re
import string
from nltk.stem import  WordNetLemmatizer
from nltk import  word_tokenize
from nltk.corpus import stopwords

stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return token not in stop_words_ and token not in list(string.punctuation) and len(token)>2

def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'","",text)
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.replace("nbps","")
    clean_text = [wn.lemmatize(word,pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)

In [128]:
df_all['text'] = df_all['text'].apply(clean_txt)

C:\conda_temp\ipykernel_12192\2433368501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['text'] = df_all['text'].apply(clean_txt)


In [178]:
# Clean job corpus

df_all['text']

0       staff software engineer gcp aws bangalore beng...
1       job opportunity senior software engineer relia...
2       senior software engineer sql oracle dba aix ba...
3       senior software engineer net pdf microservices...
4       team lead senior software engineer php php rea...
                              ...                        
4976    software engineer iii computer science python ...
4977    software engineer backend python golang comput...
4978    software engineer iii computer science python ...
4979    software engineer net net angular html css pun...
4980    software engineer iii java backend java spring...
Name: text, Length: 4981, dtype: object

## TF-IDF ( Term Frequency - Inverse Document Frequency )

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_job = tfidf_vectorizer.fit_transform((df_all['text']))
tfidf_job

<4981x3187 sparse matrix of type '<class 'numpy.float64'>'
	with 65419 stored elements in Compressed Sparse Row format>

In [138]:
users = {'user_id':[1,2,3,4],'text':['python django machine learning','java spring boot','react sql aws','html css javascript angular',]}

In [139]:
df_user = pd.DataFrame(users)

In [140]:
df_user

,user_id,text
0,1,python django machine learning
1,2,java spring boot
2,3,react sql aws
3,4,html css javascript angular


In [149]:
u=1
index = np.where(df_user['user_id'] == u)[0][0]
user_q = df_user.iloc[[index]]
user_q

,user_id,text
0,1,python django machine learning


## Recommender system

In [150]:
# Computing cosine similarity

from sklearn.metrics.pairwise import cosine_similarity
user_tfidf = tfidf_vectorizer.transform(user_q['text'])
cosine_sim_tfidf = map(lambda x: cosine_similarity(user_tfidf,x),tfidf_job)

In [151]:
output = list(cosine_sim_tfidf)

In [152]:
def get_job_recommendation(top,df_all,scores):
    recommendation = pd.DataFrame(columns = ['user_id','title','score'])
    count = 0
    for i in top:
        recommendation.at[count,'user_id'] = u
        recommendation.at[count,'title'] = df_all['Job Title'][i]
        recommendation.at[count,'score'] = scores[count]
        count+=1
    return recommendation

In [158]:
top = sorted(range(len(output)),key=lambda i: output[i],reverse=True)[:10]
list_scores = [output[i][0][0] for i in top]
df_recommend =get_job_recommendation(top,df_all,list_scores)

In [176]:
df_recommend

,user_id,title,score
0,1,Software Engineer: Backend (Python & Django),0.464483
1,1,Software Engineer: Backend (Python & Django),0.464483
2,1,ML Software Engineer at Cargill,0.440224
3,1,"Python (Software Engineer, Senior Software Eng...",0.420638
4,1,"Python Software Engineer, Senior Software Engi...",0.402302
5,1,Engineer Software Development - Python,0.391
6,1,Engineer Software Development - Python,0.386467
7,1,Software Engineer - Django,0.356061
8,1,Senior Backend Software Engineer ( Python / Dj...,0.33877
9,1,Software Engineer,0.336175
